In [1]:
%cd ..

/Users/shpotes/Developer/factored-datathon-2023-almond


In [2]:
import pandas as pd
import uuid

from src.modelling.topics.topic_extractor import TopicExtractionConfig, TopicExtractor
from src.modelling.topics.class_tf_idf import ClassTfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
import bs4

def clean_html(html_text):
  soup = bs4.BeautifulSoup(html_text)
  text = soup.get_text()
  return text

In [11]:
df = pd.read_csv('raw_data/B0012Y0ZG2_reviews.csv')

df

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,review_id,vote,image
0,5.0,False,"09 11, 2009",A1L4ZAG36ZO15M,B0012Y0ZG2,{'Size:': ' 6.8 oz'},Birmingham Man,Best general purpose shampoo on the market. My...,Penhaligon's Blenheim Bouquet Shampoo - GREAT !,1252627200,59e1161a-4a7a-4e49-a5e8-0fa78560a394,0,NaN
1,5.0,True,"08 20, 2009",A23JI9AN3N4GFK,B0012Y0ZG2,{'Size:': ' 106'},J. P. Lewis,I've used Bellmira Herbaflor Herbal Baths for ...,Fabulous Bubbles...,1250726400,257569be-b2d5-4457-aeb3-e3026f542890,0,NaN
2,5.0,False,"08 3, 2009",A2VABJ7S3OTH4K,B0012Y0ZG2,{'Size:': ' 263'},Robin DeLu,I purposely went online to ebay to try to find...,Thrilled to find this discontinued product on ...,1249257600,1410a477-8caf-4a0e-942b-4e3c6af2aa3f,0,NaN
3,5.0,False,"07 8, 2009",A1CUV4MCTJIVPU,B0012Y0ZG2,{'Size:': ' 6.8 oz'},Lindsey L. Walczak,"I have extremely long hair, so finding a good ...",awesome,1247011200,3af644b7-d2f3-4865-a32b-f791bb3fa7d3,0,NaN
4,5.0,True,"07 6, 2009",AV9DG8YERA6HC,B0012Y0ZG2,{'Size:': ' 247'},Lois N. Herlihy,"I love this product , it has been discontinued...",LOVE THIS PRODUCT,1246838400,311322ea-582a-4dc6-83d3-bed24d49f413,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2227,5.0,True,"05 22, 2018",A3PFH26EDV59NX,B0012Y0ZG2,{'Size:': ' 401'},Ronale Hall,Love love love,Five Stars,1526947200,3f0f9a7f-2635-43f7-9491-23af91506565,0,NaN
2228,3.0,True,"05 20, 2018",A3KIR33PSLNWXQ,B0012Y0ZG2,{'Size:': ' 8.5oz'},Andrea Mendoza,Feels great on my skin the only problem is the...,Too watery,1526774400,9bab8d3c-ea53-4ac2-9014-49dceabb0baa,0,NaN
2229,3.0,True,"05 19, 2018",ARPSCXPD7FYZ4,B0012Y0ZG2,{'Size:': ' 8.5oz'},thomas vanmeter,I did not want to give a bad review for the Sh...,Incomplete Decision,1526688000,43558b5b-f728-4bd9-8456-4a3f5eb99029,0,NaN
2230,5.0,True,"05 19, 2018",ARPSCXPD7FYZ4,B0012Y0ZG2,{'Size:': ' Small'},thomas vanmeter,"Great product, I had no issues and love the wa...",Great Product,1526688000,1bdaacb7-bd73-4899-8707-3f2bd480a54a,0,NaN


In [13]:
reviews = pd.DataFrame({
    "text": df.reviewText.apply(clean_html),
    "id": df.review_id,
    "topic": df.overall,  # experimental!
    "product_id": df.asin,
    "score": df.overall, 
    "vote": df.vote,
})


reviews

/var/folders/53/_11mnb7s3kvbc6qgkwv_5rh00118v9/T/ipykernel_43478/1107407270.py:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = bs4.BeautifulSoup(html_text)


,text,id,topic,product_id,score,vote
0,Best general purpose shampoo on the market. My...,59e1161a-4a7a-4e49-a5e8-0fa78560a394,5.0,B0012Y0ZG2,5.0,0
1,I've used Bellmira Herbaflor Herbal Baths for ...,257569be-b2d5-4457-aeb3-e3026f542890,5.0,B0012Y0ZG2,5.0,0
2,I purposely went online to ebay to try to find...,1410a477-8caf-4a0e-942b-4e3c6af2aa3f,5.0,B0012Y0ZG2,5.0,0
3,"I have extremely long hair, so finding a good ...",3af644b7-d2f3-4865-a32b-f791bb3fa7d3,5.0,B0012Y0ZG2,5.0,0
4,"I love this product , it has been discontinued...",311322ea-582a-4dc6-83d3-bed24d49f413,5.0,B0012Y0ZG2,5.0,0
...,...,...,...,...,...,...
2227,Love love love,3f0f9a7f-2635-43f7-9491-23af91506565,5.0,B0012Y0ZG2,5.0,0
2228,Feels great on my skin the only problem is the...,9bab8d3c-ea53-4ac2-9014-49dceabb0baa,3.0,B0012Y0ZG2,3.0,0
2229,I did not want to give a bad review for the Sh...,43558b5b-f728-4bd9-8456-4a3f5eb99029,3.0,B0012Y0ZG2,3.0,0
2230,"Great product, I had no issues and love the wa...",1bdaacb7-bd73-4899-8707-3f2bd480a54a,5.0,B0012Y0ZG2,5.0,0


In [14]:
# 

topic_extraction_config = TopicExtractionConfig(
    vectorizer_model = CountVectorizer(ngram_range=(2, 4), stop_words='english'),
    ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
)

topic_extractor = TopicExtractor(topic_extraction_config)

In [15]:
topics = topic_extractor(reviews)

In [22]:
topics[1].representative_examples

["I really like that this product is paraben-free, but the execution falls flat on its face. The bottle is tiny for the price. Money-to-value ratio is terrible. To add to that, the actual product itself is very thin and feels watered-down. To be fair, that may be the lack of additives, but it leads to over-use of the gel to get adequate coverage and actually feel clean after using it. To top it off, it really doesn't smell very good, either.",
 "I agree with several of the other reviewers. Feels as though Im just rubbing banana scented water through my hair and after I shower, my hair feels greesy (like when I don't use any shampoo) so I wonder if there is something fishy going on here. Read the other reviews and you'll noticed most of them have nothing to do with shampoo. DO NOT trust.",
 "I really like using organic products and this product had awesome reviews. Ive been seeing a dermatologist for my psoriasis and received a much of topical cream and even steroid injections. So I wan